In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
import xgboost as xgb

# Pipeline and scaling preprocessing will be used for models that are sensitive
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [103]:
train=pd.read_csv('/content/train_new.csv')
test=pd.read_csv('/content/test_new.csv')
test_old=pd.read_csv('/content/test.csv')

#check the shape of train and test data
print(train.shape)
print(test.shape)

(1460, 223)
(1459, 222)


In [104]:
#drop the first column which is the redundant
train = train.drop(train.columns[[0]], axis=1)
test = test.drop(test.columns[[0]], axis=1)

#check the shape of train and test data
print(train.shape)
print(test.shape)

(1460, 222)
(1459, 221)


In [105]:
test

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,EnclosedPorch,ExterCond,ExterQual,Fence,FireplaceQu,Fireplaces,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageYrBlt,GrLivArea,HalfBath,HeatingQC,KitchenAbvGr,KitchenQual,LandSlope,LotArea,LotFrontage,LotShape,LowQualFinSF,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,ID
0,11.818680,0.000000,0.0,0.730463,1.194318,1.820334,1.540963,10.105326,7.397498,2.055642,1.540963,0.000000,0.000000,1.820334,8.780689,0.730463,0.0,1.820334,1.540963,1.194318,1.540963,0.000000,0.730463,2.259674,11.259868,0.730463,2.055642,1.540963,2.055642,14.121376,11.818680,0.000000,1.820334,0.730463,1.540963,0.000000,20.479373,6.221214,1.540963,0.0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN
1,12.943735,0.000000,0.0,0.730463,1.540963,1.820334,1.540963,11.901094,0.000000,0.000000,2.259674,0.000000,0.000000,1.820334,9.752379,0.730463,0.0,1.820334,1.540963,1.820334,1.540963,0.000000,0.730463,2.259674,9.118181,0.730463,2.055642,1.540963,2.055642,14.116605,12.943735,0.730463,1.820334,0.730463,1.194318,0.000000,21.327220,6.244956,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN
2,11.916131,11.151348,0.0,0.730463,1.540963,1.820334,1.540963,11.476685,0.000000,1.194318,2.259674,0.000000,0.000000,1.194318,7.293500,0.730463,0.0,1.820334,1.540963,1.194318,2.055642,0.730463,1.194318,2.259674,10.179489,1.194318,2.055642,0.000000,2.055642,14.178149,13.551272,0.730463,1.194318,0.730463,1.540963,0.000000,21.196905,6.073289,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN
3,11.910125,11.062536,0.0,0.730463,1.540963,1.820334,1.540963,10.749651,0.000000,1.194318,2.259674,0.000000,0.000000,1.820334,9.207511,0.730463,0.0,1.820334,1.540963,1.820334,1.194318,0.730463,1.194318,2.259674,10.116035,1.194318,2.055642,0.000000,2.055642,14.179714,13.504453,0.730463,0.000000,0.730463,1.194318,0.000000,19.865444,6.172972,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN
4,12.833625,0.000000,0.0,0.730463,1.194318,1.820334,1.540963,8.720170,0.000000,0.000000,2.259674,0.000000,0.000000,1.194318,12.172900,0.730463,0.0,1.820334,1.194318,1.820334,1.540963,0.000000,1.194318,2.259674,10.302477,1.194318,2.055642,1.194318,2.055642,14.170316,12.833625,0.000000,0.000000,0.730463,1.194318,0.000000,17.257255,5.093857,0.000000,0.0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,10.496872,10.496872,0.0,0.730463,1.540963,1.820334,1.540963,0.000000,0.000000,2.259674,2.259674,0.000000,0.000000,1.820334,10.496872,0.730463,0.0,1.820334,1.540963,1.820334,1.540963,0.000000,0.730463,2.259674,0.000000,0.000000,1.540963,0.730463,1.540963,0.000000,12.374860,0.730463,1.194318,0.730463,1.540963,0.000000,14.081426,3.932510,1.540963,0.

In [0]:
#save ID 
train_ID = train['ID']
test_ID = test_old['Id']
# FIx this bug
train.drop("ID", axis = 1, inplace = True)
test.drop("ID", axis = 1, inplace = True)

In [107]:
train_ID

0          1
1          2
2          3
3          4
4          5
        ... 
1455    1456
1456    1457
1457    1458
1458    1459
1459    1460
Name: ID, Length: 1460, dtype: int64

In [0]:
# Now we use the train to do the model building
target = train["SalePrice"]
# Define the variable
df_1 = train.copy()
df_1.drop("SalePrice", axis = 1, inplace = True)
train_var = df_1

# train_var,target,test

In [109]:
print(target.shape)
print(train_var.shape)

(1460,)
(1460, 220)


# Model 1: Nerual Network for Regression

In [0]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

In [86]:
# Build the NN network
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train_var.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               28288     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 193,153
Trainable params: 193,153
Non-trainable params: 0
_________________________________________________________________


In [0]:
# set a check point 
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [88]:
NN_model.fit(train_var, target, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)




Train on 1168 samples, validate on 292 samples
Epoch 1/500





1168/1168 [==============================] - 5s 4ms/step - loss: 179419.9677 - mean_absolute_error: 179419.9677 - val_loss: 173439.4253 - val_mean_absolute_error: 173439.4253

Epoch 00001: val_loss improved from inf to 173439.42530, saving model to Weights-001--173439.42530.hdf5
Epoch 2/500
1168/1168 [==============================] - 0s 213us/step - loss: 99702.4468 - mean_absolute_error: 99702.4468 - val_loss: 47439.6064 - val_mean_absolute_error: 47439.6064

Epoch 00002: val_loss improved from 173439.42530 to 47439.60643, saving model to Weights-002--47439.60643.hdf5
Epoch 3/500
1168/1168 [==============================] - 0s 171us/step - loss: 46534.9655 - mean_absolute_error: 46534.9655 - val_loss: 44419.6706 - val_mean_absolute_error: 44419.6706

Epoch 00003: val_loss improved from 47439.60643 to 44419.67059, saving model to Weights-003--44419.67059.hdf5
Epoch 4/500
1168/1168 [==============================] - 0s 

In [0]:
# Load wights file of the best model :
wights_file = 'Weights-172--17815.28280.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [0]:
# Make prediction on our test set
predictions = NN_model.predict(test)

In [112]:
def make_submission(prediction, sub_name, ID):
  my_submission = pd.DataFrame({'Id':ID,'SalePrice':prediction})
  my_submission.to_csv('{}.csv'.format(sub_name),index=False)
  print('A submission file has been made')

make_submission(predictions[:,0],'submission(NN)', test_ID)

A submission file has been made


In [111]:
test_ID

# This result return us the rank: 2000(37%)

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64